In [0]:
!wget -d https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.pt.zip
!unzip -o wiki.pt.zip

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

from keras.layers import Dense, Embedding, Bidirectional, GRU
from numpy import array
import numpy as np
import codecs
from tqdm import tqdm
from keras.preprocessing.text import text_to_word_sequence
tqdm.pandas()

In [0]:
# Esta função nos permite usar um Word Embedding pré-treinado
# em um outro conjunto de dados. Como nosso dataset nesta tarefa é
# muito pequeno, pode ser uma boa ideia usar um word embedding pré-treinado

def load_fasttext(word_index, max_features):    
    embeddings_index = {}
    f = codecs.open("wiki.pt.vec", encoding='utf-8')
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('%s word vectors encontrados' % len(embeddings_index))
    
    words_not_found = []
    
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if (embedding_vector is not None) and len(embedding_vector) > 0:
            # palavras nao encontradas no embedding permanecem com valor nulo
            embedding_matrix[i] = embedding_vector
        else:
            words_not_found.append(word)
    
    print('quantidade de word embeddings nulas: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    
    return embedding_matrix

In [0]:
def clean_text(text):
    text = text.replace('"', ' ')
    text = text.replace('\'', ' ')    
    text = text.replace('<', ' ')
    text = text.replace('>', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('*', ' ')
    text = text.replace('\\', ' ')
    text = text.strip()
    return text.lower()

# gera uma sequência de palavras, a partir de uma sequência anterior
# nós usaremos esta função para testar nosso modelo
def generate_seq(model, tokenizer, max_length, in_text, n_words):
	
	for _ in range(n_words):
		# cria uma sequência de valores inteiros a partir do texto de entrada
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		
        # fixa o tamanho das sequências
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		
        # estima a probabilidade para cada palavra da sequência dada
		yhat = model.predict_classes(encoded, verbose=0)
		
        # esta palavra está presente no índice de palavras do seu vocabulário?
        # Se a palavra já foi observada, então vamos
        # incluí-la na frase que está sendo gerada.
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# concatena a palavra na sequência sendo gerada
		in_text += ' ' + out_word
	return in_text

# Treinando o modelo

In [18]:
# nosso pequeno dataset
data = []
with open("dataset.txt", mode="r", encoding="latin") as file:
	for line in file:		
		data.append(clean_text(line))

data = "\n".join(data)


tokenizer = Tokenizer()

tokenizer.fit_on_texts([data])

vocab_size = len(tokenizer.word_index) + 1
print('Tamanho do vocabulário: %d' % vocab_size)

# cria uma sequência de valores inteiros a partir do texto de entrada
encoded = tokenizer.texts_to_sequences([data])[0]

# Aqui, cada palavra estimada está condicionada àquelas palavras
# que aparecem antes dela na sequência. Para estimar uma palavra específica,
# nós consideramos as 2 palavras que vêm antes dela.
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)


print('número de sequências: %d' % len(sequences))

# apenas para assegurar que teremos sequências sempre com um mesmo tamanho
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

print('Tamanho máximo da sequência: %d' % max_length)

# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)


word_index = tokenizer.word_index
max_features = len(word_index)+1

# carrega nosso word embedding pré-treinado
embedding_matrix = load_fasttext(word_index, max_features)

model = Sequential()
model.add(Embedding(max_features, 300, weights=[embedding_matrix], input_length=max_length-1, trainable=False))
model.add(Bidirectional(GRU(50)))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

# treina o nosso modelo.
model.fit(X, y, epochs=500)


150529it [00:22, 6266.10it/s]
151215it [00:22, 6432.32it/s]
151896it [00:22, 6538.54it/s]
152556it [00:22, 6554.93it/s]
153237it [00:22, 6627.07it/s]
153930it [00:22, 6714.33it/s]
154603it [00:23, 6680.09it/s]
155282it [00:23, 6710.12it/s]
155954it [00:23, 6556.59it/s]
156640it [00:23, 6642.27it/s]
157323it [00:23, 6696.57it/s]
157994it [00:23, 6690.14it/s]
158686it [00:23, 6755.48it/s]
159382it [00:23, 6814.00it/s]
160064it [00:23, 6662.36it/s]
160751it [00:24, 6722.27it/s]
161433it [00:24, 6749.38it/s]
162109it [00:24, 6745.82it/s]
162785it [00:24, 6515.66it/s]
163453it [00:24, 6562.29it/s]
164143it [00:24, 6659.36it/s]
164811it [00:24, 6636.94it/s]
165522it [00:24, 6771.51it/s]
166225it [00:24, 6846.14it/s]
166915it [00:24, 6861.15it/s]
167602it [00:25, 6824.81it/s]
168294it [00:25, 6848.85it/s]
168980it [00:25, 6728.46it/s]
169654it [00:25, 6465.98it/s]
170323it [00:25, 6530.75it/s]
170990it [00:25, 6570.17it/s]
171670it [00:25, 6637.35it/s]
172365it [00:25, 6727.48it/s]
173058it 

592107 word vectors encontrados
quantidade de word embeddings nulas: 23
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2, 300)            290700    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100)               105300    
_________________________________________________________________
dense_2 (Dense)              (None, 969)               97869     
Total params: 493,869
Trainable params: 203,169
Non-trainable params: 290,700
_________________________________________________________________
Epoch 1/500
3337/3337 [==============================] - 3s 1ms/step - loss: 6.3679
Epoch 2/500
3337/3337 [==============================] - 2s 553us/step - loss: 5.7995
Epoch 3/500
3337/3337 [==============================] - 2s 553us/step - loss: 5.5450
Epoch 4/500
3337/3337 [==============================] - 2s 545us/

# Testando o nosso modelo
Digite uma sequência de duas palavras no campo indicado abaixo e o modelo vai gerar uma sentença a partir delas. Tente algo como **"minha terra"**

In [0]:
while True:
    sentence = input("\ninput> ")

    if sentence == "exit":
        break

    print("\n output: ", generate_seq(model, tokenizer, max_length-1, sentence, 15))


input> um dia desses

 output:  um dia desses num desses encontros casuais talvez a gente aprende se você quiser alguém em quem confiar

input> eu só peço

 output:  eu só peço a deus um pouco de malandragem pois sou criança e não aprendi a amar eu

input> minha terra

 output:  minha terra tem palmeiras onde canta o sabiá as aves que aqui gorjeiam não gorjeiam como lá

input> quem com o ferro

 output:  quem com o ferro fere não sabe amar mas quem sente muito cala quem quer dizer quanto sente fica

input> pra ser sincero

 output:  pra ser sincero não espero que você nunca viu agora eu vejo aquele beijo era mesmo o fim

input> mudaram as estações

 output:  mudaram as estações e nada mudou mas eu sei que faço isso pra esquecer eu deixo a onda
